# **Autoencoder**

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../')

from src.autoencoder.autoencoder import Autoencoder

In [ ]:
layer_sizes = [784, 256, 128, 64, 128, 256, 784] 
autoencoder = Autoencoder(layer_sizes)

x = torch.randn((64, 784))

reconstructed = autoencoder(x)
print(reconstructed.shape)